In [ ]:
"""
#TensorFlow 2.10 was the last TensorFlow release that supported GPU on native-Windows.
!pip install "tensorflow-gpu<2.11"  
!pip install --upgrade pip
!pip install pandas
!pip install nltk
!pip install rake_nltk 
!pip install -U scikit-learn
!pip install transformers
!pip install matplotlib
!pip install numpy
!pip install seaborn
!pip install pdfminer.six
!pip install pdfrw
# For error : AttributeError: module 'keras.engine.data_adapter' has no attribute 'expand_1d'
!pip install --upgrade git+https://github.com/huggingface/transformers.git"""

In [ ]:
import pandas as pd 
df_model=pd.read_csv('../csv/NL_document_targets_deg_to_ratio_filtered.csv',sep=';',index_col=[0])

print (f'Number of encoded columns: {len(df_model.columns)-2}')
encoded_columns=len(df_model.columns)-2

In [ ]:
df_model.head()

## Determine clusters from the first (n) ohe columns

In [ ]:
y_1 = pd.get_dummies(df_model['1'])
y_2 = pd.get_dummies(df_model['2'])
y_3 = pd.get_dummies(df_model['3'])
y_4 = pd.get_dummies(df_model['4'])
y_5 = pd.get_dummies(df_model['5'])
y_6 = pd.get_dummies(df_model['6'])
y_7 = pd.get_dummies(df_model['7'])
y_8 = pd.get_dummies(df_model['8'])
y_9 = pd.get_dummies(df_model['9'])
y_10 = pd.get_dummies(df_model['10'])

y=pd.concat([y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10],axis=1,join='inner') 

y = y.groupby(level=0,axis=1).sum()
y_unique = y.loc[:,~y.columns.duplicated()].copy() 

y=y_unique 
y.head(-3)


In [ ]:
from sklearn.cluster import KMeans

def ideal_n_clusters():
    Sum_of_squared_distances = []
    K = range(1,100)
    for k in K:
        km = KMeans(n_clusters=k)
        km = km.fit(y)
        Sum_of_squared_distances.append(km.inertia_)

    import matplotlib.pyplot as plt
    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

#ideal_n_clusters()

In [ ]:
from sklearn.decomposition import PCA #Principal Component Analysis
import numpy as np

kmeans = KMeans(n_clusters=100)
kmeans.fit(y)

#Find which cluster each data-point belongs to
clusters = kmeans.predict(y)

#Add the cluster vector to our DataFrame, y
y["cluster"] = clusters
df_model["cluster"] = clusters
 
#plotX is a DataFrame containing 5000 values sampled randomly from X
plot_y = pd.DataFrame(np.array(y))

#Rename plotX's columns since it was briefly converted to an np.array above
plot_y.columns = y.columns

In [ ]:
y.head()

In [ ]:
#sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding
from sklearn.preprocessing import StandardScaler #used for 'Feature Scaling'

#plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
#PCA with one principal component
pca_1d = PCA(n_components=1)

#PCA with two principal components
pca_2d = PCA(n_components=2)

#PCA with three principal components
pca_3d = PCA(n_components=3)

#PCA with 4 principal components
pca_4d = PCA(n_components=4)

#PCA with 4 principal components
pca_5d = PCA(n_components=5)

In [ ]:
#This DataFrame holds that single principal component mentioned above
PCs_1d = pd.DataFrame(pca_1d.fit_transform(plot_y.drop(["cluster"], axis=1)))

#This DataFrame contains the two principal components that will be used
#for the 2-D visualization mentioned above
PCs_2d = pd.DataFrame(pca_2d.fit_transform(plot_y.drop(["cluster"], axis=1)))

#And this DataFrame contains three principal components that will aid us
#in visualizing our clusters in 3-D
PCs_3d = pd.DataFrame(pca_3d.fit_transform(plot_y.drop(["cluster"], axis=1)))

#And this DataFrame contains three principal components that will aid us
#in visualizing our clusters in 4-D
PCs_4d = pd.DataFrame(pca_4d.fit_transform(plot_y.drop(["cluster"], axis=1)))

#And this DataFrame contains three principal components that will aid us
#in visualizing our clusters in 4-D
PCs_5d = pd.DataFrame(pca_5d.fit_transform(plot_y.drop(["cluster"], axis=1)))

#Rename the columns of these newly created DataFrames:
PCs_1d.columns = ["PC1_1d"]

#"PC1_2d" means: 'The first principal component of the components created for 2-D visualization, by PCA.'
#And "PC2_2d" means: 'The second principal component of the components created for 2-D visualization, by PCA.'
PCs_2d.columns = ["PC1_2d", "PC2_2d"]

PCs_3d.columns = ["PC1_3d", "PC2_3d", "PC3_3d"]
PCs_4d.columns = ["PC1_4d", "PC2_4d", "PC3_4d", "PC4_4d"]
PCs_5d.columns = ["PC1_4d", "PC2_4d", "PC3_4d", "PC4_4d", "PC5_5d"]

#plot_y = pd.concat([plot_y,PCs_1d,PCs_2d,PCs_3d], axis=1, join='inner')
#plot_y = pd.concat([plot_y,PCs_1d,PCs_2d,PCs_3d,PCs_4d], axis=1, join='inner')
plot_y = pd.concat([plot_y,PCs_1d,PCs_2d,PCs_3d,PCs_4d,PCs_5d], axis=1, join='inner')

# create one new column for plotX so that we can use it for 1-D visualization.
plot_y["dummy"] = 0

#Note that all of the DataFrames below are sub-DataFrames of 'plotX'.
#This is because we intend to plot the values contained within each of these DataFrames.

cluster0 = plot_y[plot_y["cluster"] == 0]
cluster1 = plot_y[plot_y["cluster"] == 1]
cluster2 = plot_y[plot_y["cluster"] == 2]
cluster3 = plot_y[plot_y["cluster"] == 3]
cluster4 = plot_y[plot_y["cluster"] == 4]

#This is needed so we can display plotly plots properly
init_notebook_mode(connected=True)

In [ ]:
#Instructions for building the 2-D plot
# https://www.kaggle.com/code/minc33/visualizing-high-dimensional-clusters/notebook
#trace1 is for 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["PC1_2d"],
                    y = cluster0["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 is for 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["PC1_2d"],
                    y = cluster1["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 is for 'Cluster 2'
trace3 = go.Scatter(
                    x = cluster2["PC1_2d"],
                    y = cluster2["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3]

title = "Visualizing Clusters in Two Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [ ]:
#Instructions for building the 3-D plot

#trace1 is for 'Cluster 0'
trace1 = go.Scatter3d(
                    x = cluster0["PC1_3d"],
                    y = cluster0["PC2_3d"],
                    z = cluster0["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 is for 'Cluster 1'
trace2 = go.Scatter3d(
                    x = cluster1["PC1_3d"],
                    y = cluster1["PC2_3d"],
                    z = cluster1["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 is for 'Cluster 2'
trace3 = go.Scatter3d(
                    x = cluster2["PC1_3d"],
                    y = cluster2["PC2_3d"],
                    z = cluster2["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

#trace3 is for 'Cluster 3'
trace4 = go.Scatter3d(
                    x = cluster3["PC1_3d"],
                    y = cluster3["PC2_3d"],
                    z = cluster3["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 3",
                    marker = dict(color = 'rgba(128, 10, 200, 0.8)'),
                    text = None)

#trace3 is for 'Cluster 4'
trace5 = go.Scatter3d(
                    x = cluster4["PC1_3d"],
                    y = cluster4["PC2_3d"],
                    z = cluster4["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 4",
                    marker = dict(color = 'rgba(2, 128, 0, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3,trace4,trace5]

title = "Visualizing Clusters in Three Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

For this we need the  Jaccard Distance 

Because eucledian distance also takes into account the 0 correlation

Jaccard counts the 1 correlations

In [ ]:
def jaccard():
    import scipy

    jaccard = scipy.spatial.distance.cdist(y, y,  
                                        metric='jaccard')
    user_distance = pd.DataFrame(jaccard, columns=y.index.values,  
                                index=y.index.values)

    user_distance.head()

## Make the full text shorter with NLTK

- Get ranked phrases with Rake_NLTK<br>
- Select only these terms which identify as NL with fasttext


In [ ]:
# FASTTEXT LANGUAGE detection 
import fasttext as ft

# Load the pretrained model
ft_model = ft.load_model("../preprocessing/lid.176.ftz")

def fasttext_language_predict(text, model = ft_model):

  text = text.replace('\n', " ")
  prediction = model.predict([text])

  return prediction

In [ ]:
# Make the text in dataframe shorter 
# And replace df_model['text'] with this shorter condensed version

def shorten_text():
    from nltk.corpus import stopwords

    # Add custom stopwords
    stopwords = stopwords.words('dutch')

    custom_stopwords=['per','waar','waarvoor','wegens','wanneer','gevolg','gevolge','voorbehoud','erratum','correctie','sommige','betreffende','maatregel','procedure','stelsel','sector','organisatie','excl','aanv','adv','art','artikel','hoofdstuk','XII','XI','IX','VII','VI','V', '2020','2019','2018','2021','2022','dag','dagen','uur','uren','jaar','jaarlijks','maand','maanden','januari','februari','maart','april','mei','juni','juli','augustus','september','oktober','november','december''volgt','voordat','behoudt','beschouwd','bepaald','gedaan','leiden','zullen','gaan']
    stopwords.extend(custom_stopwords)
    

    from rake_nltk import Metric, Rake
    rake_nltk_var = Rake(language='dutch',stopwords=stopwords,include_repeated_phrases=False,ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO,min_length=1,max_length=3)
    import re

    for i in range(len(df_model)):
        full_text=df_model["text"][i]
        
        rake_nltk_var.extract_keywords_from_text(full_text)
        
        phrases_extracted = rake_nltk_var.get_ranked_phrases()

        # Keyword_extracted is a list, so split them into seperate words for the columns 
    
        phrases=set(phrases_extracted)

        # First column will be document_id, so put it as first element
        doc_keywords=[]

        # Exclude errors like 'e' as keyword
        for phrase in phrases:
            if len(phrase)>3:
                detected_language=fasttext_language_predict(phrase, model = ft_model)[0][0][0][-2:]  
                if detected_language.upper()=='NL':
                    doc_keywords.append(phrase)


        short_text=doc_keywords
        #print (len(short_text))

        #print (short_text)
        string_text=str()
        for keywords in short_text:
            string_text+=keywords
            string_text+=' '

        #print (string_text)
        df_model["text"][i]=string_text
 

shorten_text()

In [ ]:
df_clusters=df_model
df_clusters.head()

In [ ]:
# Remove the phrase/keyword columns
for i in range(1,encoded_columns+1):
    df_clusters.drop(f'{i}',axis=1,inplace=True)
df_clusters.head()

## Write the new dataframe to .csv

In [ ]:
df_clusters.to_csv('../csv/TRAIN_DS_Clusters.csv', sep=";", index=False)

## We now redefine y as target
y should now be the clusters!

In [ ]:
import pandas as pd
import tensorflow as tf
df_model=pd.read_csv('../csv/TRAIN_DS_Clusters.csv', sep=";") 
df_model.head()

*Convert y from pd Series to pd dataframe!*

In [ ]:
X = df_model["text"].astype(str).tolist()
y = df_model['cluster'].to_frame() 


In [ ]:
print (y)

In [ ]:
# Make train test val 

from sklearn.model_selection import train_test_split

# Split Train and Validation data
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=4,stratify=y)

# Keep some data for inference (testing)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=9,stratify=y)
print ('X_train',len(X_train))
print ('X_test',len(X_test))
print ('X_val',len(X_val))


In [ ]:
# Initiate GPU with maximal memory allocation

print (tf.__version__)
# Ref: https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) # Notice here
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from transformers import AutoTokenizer
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")
tokenizer = AutoTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

model = TFRobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels=len(set(y)))

In [ ]:
train_encodings = tokenizer(X_train, max_length=128, truncation=True, padding=True)
val_encodings   = tokenizer(X_val,   max_length=128, truncation=True, padding=True)
test_encodings  = tokenizer(X_test,  max_length=128, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)) 

In [ ]:
# Create AdamW optimizer

from official.nlp import optimization 
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_dataset).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 5e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
# Compile the model

loss=tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="auto",
    name="categorical_crossentropy",
)
metrics = tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.summary()

In [ ]:
# Run the model
 
tf.config.run_functions_eagerly(True)

BATCH_SIZE = 2
EPOCHS =6
history=model.fit(
    train_dataset.batch(BATCH_SIZE) ,
    epochs=EPOCHS,
    validation_data=val_dataset.batch(BATCH_SIZE)
)


In [ ]:
loss, accuracy = model.evaluate(test_dataset)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:

import matplotlib.pyplot as plt

history_dict = history.history
print(history_dict.keys())

acc = history_dict['categorical_accuracy']
val_acc = history_dict['val_categorical_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
dataset_name = 'kpmg_model_h_robberta_dutch_softmax'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))

model.save(saved_model_path, include_optimizer=False)

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [ ]:
import re

with open ('../processed_data/NL/NL_312-2019-012471.txt') as f:
    text=f.read() 
text = clean_text(text)
#print (text)

#not enough values to unpack (expected 2, got 1)
#input_ids = input_ids.unsqueeze(0)
#attention_mask = attention_mask.unsqueeze(0)

filename = 'NLP_model.sav' 
#loaded_model = pickle.load(open(filename, 'rb'))


encodings = tokenizer([text], truncation=True, padding=True)
ds = tf.data.Dataset.from_tensor_slices(dict(encodings))

print (ds)
ds=ds.batch(1, drop_remainder=True)
print(ds)
predictions = model.predict(ds)

mapping = {i: name for i, name in enumerate(y.columns)}

import numpy as np
print(mapping[np.argmax(predictions[0])])

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_loss(history):
# Use a log scale to show the wide range of values.
    plt.semilogy(history.epoch,  history.history['loss'],
               color='red', label='Train Loss')
    plt.semilogy(history.epoch,  history.history['val_loss'],
          color='green', label='Val Loss',
          linestyle="--")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
  
    plt.legend()

plot_loss(history)

In [ ]:
!pip install seaborn
from sklearn.metrics import roc_curve,confusion_matrix,auc
import seaborn as sns

def plot_cm(y_true, y_pred, title):
    ''''
    input y_true-Ground Truth Labels
          y_pred-Predicted Value of Model
          title-What Title to give to the confusion matrix
    
    Draws a Confusion Matrix for better understanding of how the model is working
    
    return None
    
    '''
    
    figsize=(10,10)
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    plt.title(title)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)
 
val_dataset=val_dataset.batch(1, drop_remainder=True) 
y_predict=model.predict(val_dataset, verbose=1)

In [ ]:
print (y_predict[0])
y_predict[float(y_predict[0])> 0.5] = 1
y_predict[y_predict <= 0.5] = 0
plot_cm(y_val, y_predict, 'Performance-Confusion Matrix')
plot_cm()
